In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import odeint

In [2]:
# not confirmed data
mumax_Glc_inh = 0.3 # h^-1
kS_Glc_inh = 0.1
kS_O2_inh = 30
kS_Eth_inh = 0.1
kI_Eth_inh = 46
Y_XGlc_inh = 0.15
Y_CO2Glc_inh = 0.5
Y_EthGlc_inh = 10.0e-3
Y_H2OGlc_inh = 1
Y_QGlc_inh = 1.0e-3
RQ_Glc_inh = 1
RQ_Eth_inh = 1
KF_rx = 1 #placeholder for temperature-dependent correction factor

# confirmed data
mumax_Glc_aerob = 0.5  # h^-1
mumax_Glc_crab = 0.34  # h^-1
mumax_Glc_anaerob = 0.25  # h^-1
mumax_Eth_aerob = 0.36  # h^-1
kS_Glc_aerob = 0.1  # kg/m^3
kS_Glc_crab = 0.001  # kg/m^3
kS_Glc_anaerob = 0.1  # kg/m^3
kS_O2_aerob = 30  # Vol-%
kS_O2_crab = 30  # Vol-%
kS_Eth_aerob = 0.1  # kg/m^3
kI_Glc_aerob = 0.1  # kg/m^3
kI_Eth_aerob = 46  # kg/m^3
kI_Eth_crab = 46  # kg/m^3
kI_O2_anaerob = 10  # Vol-%
T1 = 0.2  # h
T2 = 0.2  # h
T3 = 0.2  # h
T4 = 0.2  # h
rd = 0.005  # h^-1
Y_XGlc_aerob = 0.56  # g/g
Y_XGlc_crab = 0.14  # g/g
Y_XGlc_anaerob = 0.15  # g/g
Y_XEth_aerob = 0.67  # g/g
Y_CO2Glc_aerob = 1.46  # g/g
Y_CO2Glc_crab = 0.53  # g/g
Y_CO2Glc_anaerob = 0.48  # g/g
Y_CO2Eth_aerob = 1.9  # g/g
Y_EthGlc_anaerob = 0.51  # g/g
Y_EthGlc_crab = 0.31  # g/g
RQ_Glc_aerob = 1  # g/g
RQ_Glc_crab = 9  # g/g
RQ_Eth_aerob = 0.66  # g/g
Y_H2OGlc_aerob = 6  # mol/mol
Y_H2OGlc_crab = 0.25  # mol/mol
Y_H2OEth_aerob = 3  # mol/mol
Y_QGlc_aerob = 15.6e-3  # J/kg
Y_QGlc_crab = 1.08e-3  # J/kg
Y_QGlc_anaerob = 0.46e-3  # J/kg
Y_QEth_aerob = 29.7e-3  # J/kg

In [3]:
def create_main_plot(tspan, c_Glc, c_Eth, XvXd):
    # Create plots using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan * 60, y=c_Glc, name='Glucose'))
    fig.add_trace(go.Scatter(x=tspan * 60, y=c_Eth, name='Ethanol'))
    fig.add_trace(go.Scatter(x=tspan * 60, y=Xv + Xd, name='Cell mass'))
    fig.update_layout(
        title='Behaviour over Time',
        xaxis_title='Time (minutes)',
        yaxis_title='Concentration (g/L)',
        legend=dict(x=1, y=1)
    )
    
    # Set slider parameters
    slider_params = {
        'steps': [{'args': [[selected_time], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}],
               'label': f'{selected_time:.2f}',
               'method': 'animate'}
              for selected_time in tspan * 60]
    }
    
    # Add slider configuration
    fig.update_layout(updatemenus=[{
        'buttons': [{'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, "transition": {"duration": 300,'easing': 'quadratic-in-out'}}],
                 'label': 'Play',
                 'method': 'animate'},
                {'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                 'label': 'Pause',
                 'method': 'animate'}],
        'direction': 'left',
        'pad': {'r': 0, 't': 15},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }])
    
    # Create animation frames
    frames = [go.Frame(data=[go.Scatter(x=tspan[:i + 1] * 60, y=c_Glc[:i + 1], name='Glucose'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=c_Eth[:i + 1], name='Ethanol'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=Xv[:i + 1] + Xd[:i + 1], name='Cell mass')],
                   traces=[0, 1, 2],
                   name=f'frame_{i}')
          for i in range(1, len(tspan) + 1)]
    fig.frames = frames
    
    fig.update_xaxes(range=[tspan[0] * 60, tspan[-1] * 60])
    
    # Show plot
    
    fig.show()

In [4]:
def create_sub_plot(tspan, XvXd):
    # Subplot: Biomass vs Time
    fig2 = go.Figure()
    
    fig2.add_trace(go.Scatter(x=tspan * 60, y=Xv + Xd, name='Cell mass'))
    
    fig2.update_layout(
        title='Biomass vs Time',
        xaxis_title='Time (minutes)',
        yaxis_title='Biomass Concentration (g/L)',
        legend=dict(x=1, y=1)
    )
    
    # Set slider parameters
    slider_params = {
        'steps': [{'args': [[selected_time], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}],
               'label': f'{selected_time:.2f}',
               'method': 'animate'}
              for selected_time in tspan * 60]
    }
    
    # Add slider configuration
    fig2.update_layout(updatemenus=[{
        'buttons': [{'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, "transition": {"duration": 300,'easing': 'quadratic-in-out'}}],
                 'label': 'Play',
                 'method': 'animate'},
                {'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                 'label': 'Pause',
                 'method': 'animate'}],
        'direction': 'left',
        'pad': {'r': 0, 't': 15},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }])
    
    # Create animation frames
    frames = [go.Frame(data=[go.Scatter(x=tspan[:i + 1] * 60, y=Xv[:i + 1] + Xd[:i + 1], name='Cell mass')],
                   traces=[0, 1, 2],
                   name=f'frame_{i}')
          for i in range(1, len(tspan) + 1)]
    fig2.frames = frames
    
    fig2.update_xaxes(range=[tspan[0] * 60, tspan[-1] * 60])
    
    # Show plot
    fig2.show()

In [5]:
def Aerobic_Resperation(y, t):
    SW_Glc_aerob = 1
    SW_Glc_crab = 0
    
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Inhibition
    mu_Eth_aerob = mumax_Eth_aerob*c_Eth/(kS_Eth_aerob+c_Eth)*p_O2/(kS_O2_aerob+p_O2)*KR_E # (4) Aerobic inhibited by ehtanol 
    q_Eth_aerob = 1/Y_XEth_aerob*mu_Eth_aerob # (11) specific substrate uptake rate
    q_CO2_Eth_aerob = Y_CO2Eth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (19) Carbon Dioxide Production rate
    q_O2_Eth_aerob = q_CO2_Eth_aerob*32/(44*RQ_Eth_aerob) # (22) Oxygen uptake rate 
    q_H2O_Eth_aerob = Y_H2OEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (25) Water Production rate
    q_Q_Eth_aerob = Y_QEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (29) Heat Generation
    
    # Aerobic
    mu_Glc_aerob = mumax_Glc_aerob * c_Glc / (kS_Glc_aerob + c_Glc) * p_O2 / (kS_O2_aerob + p_O2) * kI_Eth_aerob / (kI_Eth_aerob + c_Eth) * (1 - np.exp(-t / T1))
    q_Glc_aerob = 1/Y_XGlc_aerob*mu_Glc_aerob # (9) specific substrate uptake rate
    q_CO2_Glc_aerob = Y_CO2Glc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (18) Carbon Dioxide Production rate
    q_O2_Glc_aerob = q_CO2_Glc_aerob*32/(44*RQ_Glc_aerob) # (20) Oxygen uptake rate 
    q_H2O_Glc_aerob = Y_H2OGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (23) Water Production rate
    q_Q_Glc_aerob = Y_QGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (28) Heat Generation
    
    mu_total = (mu_Glc_aerob + mu_Eth_aerob) * KF_rx - rd # (30) Total biomass growth
    q_Glc_total = (q_Glc_aerob) * KF_rx # (31) Total Glucose Consumption rate
    q_Eth_total = (q_Eth_aerob) * KF_rx # (32) Total Ethanol Production rate
    q_CO2_total = (q_CO2_Glc_aerob + q_CO2_Eth_aerob) * KF_rx # (33)Total CO2 Production rate
    q_O2_total = (q_O2_Glc_aerob + q_O2_Eth_aerob) * KF_rx # (34) Total O2 Consumption rate
    q_H2O_total = (q_H2O_Glc_aerob + q_H2O_Eth_aerob) * KF_rx # (35) Total Water Production rate
    q_Q_total = 1/3600 * (q_Q_Glc_aerob + q_Q_Eth_aerob) * KF_rx # (36) Total Heat Production rate

    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        0 * q_O2_total * Xv,
        q_Q_total * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]
    return dydt

In [6]:
def Anaerobic_Inhibited(y, t):
    SW_Glc_aerob = 1
    SW_Glc_crab = 0
    
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y
    
    # Anaerobic
    mu_Glc_anaerob = mumax_Glc_anaerob*c_Glc/(kS_Glc_anaerob+c_Glc)*KR_O2 # (5) Anaerobic
    q_Glc_anaerob = 1/Y_XGlc_anaerob*mu_Glc_anaerob # (10) specific substrate uptake rate
    q_CO2_Glc_anaerob = Y_CO2Glc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (17) Carbon Dioxide Production rate
    qp_Eth_Glc_anaerob = Y_EthGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (15) Ethanol Production rate
    q_Q_Glc_anaerob = Y_QGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (26) Heat Generation
        
    # Inhibition
    mu_Eth_aerob = mumax_Eth_aerob*c_Eth/(kS_Eth_aerob+c_Eth)*p_O2/(kS_O2_aerob+p_O2)*KR_E # (4) Aerobic inhibited by ehtanol 
    q_Eth_aerob = 1/Y_XEth_aerob*mu_Eth_aerob # (11) specific substrate uptake rate
    q_CO2_Eth_aerob = Y_CO2Eth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (19) Carbon Dioxide Production rate
    q_O2_Eth_aerob = q_CO2_Eth_aerob*32/(44*RQ_Eth_aerob) # (22) Oxygen uptake rate 
    q_H2O_Eth_aerob = Y_H2OEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (25) Water Production rate
    q_Q_Eth_aerob = Y_QEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (29) Heat Generation
     
    mu_total = (mu_Glc_anaerob + mu_Eth_aerob) * KF_rx - rd # (30) Total biomass growth
    q_Glc_total = (q_Glc_anaerob ) * KF_rx # (31) Total Glucose Consumption rate
    q_Eth_total = (q_Eth_aerob - qp_Eth_Glc_anaerob) * KF_rx # (32) Total Ethanol Production rate
    q_CO2_total = ( q_CO2_Glc_anaerob + q_CO2_Eth_aerob) * KF_rx # (33)Total CO2 Production rate
    q_O2_total = (q_O2_Eth_aerob) * KF_rx # (34) Total O2 Consumption rate
    q_H2O_total = (q_H2O_Eth_aerob) * KF_rx # (35) Total Water Production rate
    q_Q_total = 1/3600 * (q_Q_Glc_anaerob + q_Q_Eth_aerob) * KF_rx # (36) Total Heat Production rate

    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        0 * q_O2_total * Xv,
        q_Q_total * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]
    return dydt



In [7]:
def Crab_Tree(y, t):
    SW_Glc_aerob = 0
    SW_Glc_crab = 1
    
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Anaerobic
    mu_Glc_anaerob = mumax_Glc_anaerob*c_Glc/(kS_Glc_anaerob+c_Glc)*KR_O2 # (5) Anaerobic
    q_Glc_anaerob = 1/Y_XGlc_anaerob*mu_Glc_anaerob # (10) specific substrate uptake rate
    q_CO2_Glc_anaerob = Y_CO2Glc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (17) Carbon Dioxide Production rate
    qp_Eth_Glc_anaerob = Y_EthGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (15) Ethanol Production rate
    q_Q_Glc_anaerob = Y_QGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (26) Heat Generation
    
    # Inhibition
    mu_Eth_aerob = mumax_Eth_aerob*c_Eth/(kS_Eth_aerob+c_Eth)*p_O2/(kS_O2_aerob+p_O2)*KR_E # (4) Aerobic inhibited by ehtanol 
    q_Eth_aerob = 1/Y_XEth_aerob*mu_Eth_aerob # (11) specific substrate uptake rate
    q_CO2_Eth_aerob = Y_CO2Eth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (19) Carbon Dioxide Production rate
    q_O2_Eth_aerob = q_CO2_Eth_aerob*32/(44*RQ_Eth_aerob) # (22) Oxygen uptake rate 
    q_H2O_Eth_aerob = Y_H2OEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (25) Water Production rate
    q_Q_Eth_aerob = Y_QEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (29) Heat Generation
     
    # Crab_Tree
    mu_Glc_crab = mumax_Glc_crab*c_Glc/(kS_Glc_crab+c_Glc)*p_O2/(kS_O2_crab+p_O2)*kI_Eth_crab/(kI_Eth_crab+c_Eth)*(1-np.exp(-t/T2))*SW_Glc_crab # (3) Crab tree
    q_Glc_crab = 1/Y_XGlc_crab*mu_Glc_crab # (8) specific substrate uptake rate
    q_CO2_Glc_crab = Y_CO2Glc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (16) Carbon Dioxide Production rate
    q_O2_Glc_crab = q_CO2_Glc_crab*32/(44*RQ_Glc_crab) # (21) Oxygen uptake rate 
    q_H2O_Glc_crab = Y_H2OGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (24) Water Production rate
    qp_Eth_Glc_crab = Y_EthGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (14) Ethanol Production rate
    q_Q_Glc_crab = Y_QGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (27) Heat Generation
    
    # Aerobic
    mu_Glc_aerob = mumax_Glc_aerob * c_Glc / (kS_Glc_aerob + c_Glc) * p_O2 / (kS_O2_aerob + p_O2) * kI_Eth_aerob / (kI_Eth_aerob + c_Eth) * (1 - np.exp(-t / T1)) # (4) aerobis
    q_Glc_aerob = 1/Y_XGlc_aerob*mu_Glc_aerob # (9) specific substrate uptake rate
    q_CO2_Glc_aerob = Y_CO2Glc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (18) Carbon Dioxide Production rate
    q_O2_Glc_aerob = q_CO2_Glc_aerob*32/(44*RQ_Glc_aerob) # (20) Oxygen uptake rate 
    q_H2O_Glc_aerob = Y_H2OGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (23) Water Production rate
    q_Q_Glc_aerob = Y_QGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (28) Heat Generation
        
    mu_total = (mu_Glc_aerob + mu_Glc_anaerob + mu_Glc_crab + mu_Eth_aerob) * KF_rx - rd # (30) Total biomass growth
    q_Glc_total = (q_Glc_aerob + q_Glc_anaerob + q_Glc_crab) * KF_rx # (31) Total Glucose Consumption rate
    q_Eth_total = (q_Eth_aerob - qp_Eth_Glc_anaerob - qp_Eth_Glc_crab) * KF_rx # (32) Total Ethanol Production rate
    q_CO2_total = (q_CO2_Glc_aerob + q_CO2_Glc_anaerob + q_CO2_Glc_crab + q_CO2_Eth_aerob) * KF_rx # (33)Total CO2 Production rate
    q_O2_total = (q_O2_Glc_aerob + q_O2_Glc_crab + q_O2_Eth_aerob) * KF_rx # (34) Total O2 Consumption rate
    q_H2O_total = (q_H2O_Glc_aerob + q_H2O_Glc_crab + q_H2O_Eth_aerob) * KF_rx # (35) Total Water Production rate
    q_Q_total = 1/3600 * (q_Q_Glc_aerob + q_Q_Glc_anaerob + q_Q_Glc_crab + q_Q_Eth_aerob) * KF_rx # (36) Total Heat Production rate

    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        0 * q_O2_total * Xv,
        q_Q_total * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]
    return dydt


In [8]:
def Anaerobic_Fermentation(y, t):
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Anaerobic
    mu_Glc_anaerob = mumax_Glc_anaerob*c_Glc/(kS_Glc_anaerob+c_Glc)*KR_O2 # (5) Anaerobic
    q_Glc_anaerob = 1/Y_XGlc_anaerob*mu_Glc_anaerob # (10) specific substrate uptake rate
    q_CO2_Glc_anaerob = Y_CO2Glc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (17) Carbon Dioxide Production rate
    qp_Eth_Glc_anaerob = Y_EthGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (15) Ethanol Production rate
    q_Q_Glc_anaerob = Y_QGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (26) Heat Generation
    
    mu_total = ( mu_Glc_anaerob) * KF_rx - rd
    q_Glc_total = (q_Glc_anaerob) * KF_rx
    q_Eth_total = (- qp_Eth_Glc_anaerob) * KF_rx
    q_CO2_total = (q_CO2_Glc_anaerob) * KF_rx
    q_O2_total = KF_rx
    q_H2O_total = KF_rx
    q_Q_total = 1/3600 * (q_Q_Glc_anaerob) * KF_rx

    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        0 * q_O2_total * Xv,
        q_Q_total * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]
    return dydt



In [9]:
def Product_inhibition(y, t):
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Calculate Product-Inhibited Specific Growth Rate
    mu_Glc_inh = mumax_Glc_inh * c_Glc / (kS_Glc_inh + c_Glc) * (1 / (P / KI_Glc_inh))
    q_Glc_inh = 1 / Y_XGlc_inh * mu_Glc_inh
    q_CO2_Glc_inh = Y_CO2Glc_inh * q_Glc_inh * (1 - Y_XGlc_inh)
    q_O2_Glc_inh = q_CO2_Glc_inh * 32 / (44 * RQ_Eth_inh)  # Corrected the equation
    q_H2O_Glc_inh = Y_H2OGlc_inh * q_Glc_inh * (1 - Y_XGlc_inh)
    qp_Eth_Glc_inh = Y_EthGlc_inh * q_Glc_inh * (1 - Y_XGlc_inh)
    q_Q_Glc_inh = Y_QGlc_inh * q_Glc_inh * (1 - Y_XGlc_inh)
    
    # Inhibition
    mu_Eth_aerob = mumax_Eth_aerob*c_Eth/(kS_Eth_aerob+c_Eth)*p_O2/(kS_O2_aerob+p_O2)*KR_E # (4) Aerobic inhibited by ehtanol 
    q_Eth_aerob = 1/Y_XEth_aerob*mu_Eth_aerob # (11) specific substrate uptake rate
    q_CO2_Eth_aerob = Y_CO2Eth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (19) Carbon Dioxide Production rate
    q_O2_Eth_aerob = q_CO2_Eth_aerob*32/(44*RQ_Eth_aerob) # (22) Oxygen uptake rate 
    q_H2O_Eth_aerob = Y_H2OEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (25) Water Production rate
    q_Q_Eth_aerob = Y_QEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (29) Heat Generation
    
    # Anaerobic
    mu_Glc_anaerob = mumax_Glc_anaerob*c_Glc/(kS_Glc_anaerob+c_Glc)*KR_O2 # (5) Anaerobic
    q_Glc_anaerob = 1/Y_XGlc_anaerob*mu_Glc_anaerob # (10) specific substrate uptake rate
    q_CO2_Glc_anaerob = Y_CO2Glc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (17) Carbon Dioxide Production rate
    qp_Eth_Glc_anaerob = Y_EthGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (15) Ethanol Production rate
    q_Q_Glc_anaerob = Y_QGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (26) Heat Generation

    # Calculate Total Rates
    mu_total = (mu_Glc_inh + mu_Eth_aerob) * KF_rx - rd
    q_Glc_total = q_Glc_inh * KF_rx
    q_Eth_total = (q_Eth_aerob - qp_Eth_Glc_anaerob) * KF_rx
    q_CO2_total = (q_CO2_Glc_inh + q_CO2_Eth_aerob) * KF_rx
    q_O2_total = q_O2_Eth_aerob * KF_rx
    q_H2O_total = q_H2O_Eth_aerob * KF_rx
    q_Q_total = 1 / 3600 * (q_Q_Glc_anaerob + q_Q_Eth_aerob) * KF_rx

    # Define the ODEs
    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        q_Q_total * Xv,
        mu_Glc_inh * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]

    return dydt


In [18]:
mumax_Glc_batch = 0.3  # h^-1
kS_Glc_batch = 0.1
kS_O2_batch = 30
kS_Eth_batch = 0.1
kI_Eth_batch = 46
Y_XGlc_batch = 0.15
Y_CO2Glc_batch = 0.5
Y_EthGlc_batch = 10.0e-3
Y_H2OGlc_batch = 1
Y_QGlc_batch = 1.0e-3
RQ_batch = 1
k_death = 0.0138

def Batch_Culture(y, t):
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Batch Culture Growth
    mu_batch = mumax_Glc_batch * np.exp(-k_death * t)  # Fix: use mumax_Glc_batch instead of mu_max_batch
    q_Glc_batch = 1 / Y_XGlc_batch * mu_batch
    q_Eth_batch = Y_EthGlc_batch * q_Glc_batch * (1 - Y_XGlc_batch)
    q_CO2_batch = Y_CO2Glc_batch * q_Glc_batch * (1 - Y_XGlc_batch)
    q_O2_batch = q_CO2_batch * 32 / (44 * RQ_batch)
    q_H2O_batch = Y_H2OGlc_batch * q_Glc_batch * (1 - Y_XGlc_batch)
    q_Q_batch = Y_QGlc_batch * q_Glc_batch * (1 - Y_XGlc_batch)

    dydt = [
        mu_batch * Xv,
        k_death * Xv,
        -q_Glc_batch * Xv,
        -q_Eth_batch * Xv,
        q_CO2_batch * Xv,
        -q_O2_batch * Xv,
        0 * q_O2_batch * Xv,
        q_Q_batch * Xv,
        0,
        0
    ]
    return dydt